In [42]:
#948
from quickdraw import QuickDrawData, QuickDrawDataGroup
import os
import hashlib
import json

In [43]:
qd = QuickDrawData()
available = set(qd.drawing_names)
print(available)

{'helicopter', 'sword', 'castle', 'stove', 'waterslide', 'compass', 'hot tub', 'speedboat', 'parachute', 'hexagon', 'tennis racquet', 'mailbox', 'pear', 'tornado', 'toe', 'underwear', 'banana', 'computer', 'squirrel', 'rake', 'van', 'crab', 'crayon', 'hot dog', 'anvil', 'yoga', 'motorbike', 'lion', 'squiggle', 'tent', 'garden hose', 'clarinet', 'donut', 'pickup truck', 'ladder', 'rifle', 'broccoli', 'cup', 'fish', 'palm tree', 'snowflake', 'fireplace', 'oven', 'octopus', 'purse', 'saw', 'bird', 'lighthouse', 'The Mona Lisa', 'see saw', 'chandelier', 'broom', 'face', 'crown', 'binoculars', 'tractor', 'golf club', 'fire hydrant', 'hockey stick', 'owl', 'zigzag', 'hamburger', 'paper clip', 'watermelon', 'boomerang', 'dresser', 'snorkel', 'scorpion', 'frying pan', 'calendar', 'roller coaster', 'river', 'potato', 'microphone', 'giraffe', 'toilet', 'cake', 'flamingo', 'moon', 'rollerskates', 'postcard', 'candle', 'helmet', 'tree', 'panda', 'cow', 'guitar', 'ambulance', 'bear', 'canoe', 'shar

In [ ]:
#26 Label
labels = [
    "cat", "dog", "van", "swan", "flamingo", "apple",
    "angel", "candle", "book", "cup", "door", "pencil", "pig", 
    "lantern", "bat", "light bulb", "table", "necklace", "skull",
    "sword", "castle", "parachute", "hexagon", "mailbox", "fish", "octopus"
]

In [45]:
OUTDIR = "quickdraw_images_dedup"
PER_LABEL = 5000
MAX_ATTEMPTS_PER_LABEL = 100000 

In [ ]:
def drawing_uid(d):
    for attr in ("key_id", "key", "drawing_id", "id", "drawingId"):
        val = getattr(d, attr, None)
        if val:
            return str(val)
    strokes = getattr(d, "drawing", None) or getattr(d, "strokes", None)
    if strokes is None:
        img_bytes = d.image.tobytes()
        return hashlib.sha1(img_bytes).hexdigest()
    s = json.dumps(strokes, separators=(",", ":"), sort_keys=True)
    return hashlib.sha1(s.encode("utf-8")).hexdigest()

def save_drawing(d, path):
    d.image.save(path)

def fetch_unique_for_label(label, n=PER_LABEL, outdir=OUTDIR):
    os.makedirs(outdir, exist_ok=True)
    seen = set()
    saved = 0

    try:
        qdg = QuickDrawDataGroup(label, max_drawings=None)
    except Exception as e:
        print(f"Error creating QuickDrawDataGroup for '{label}': {e}")
        return saved

    available = getattr(qdg, "drawing_count", None)
    if available is not None:
        print(f"Label '{label}' has {available} drawings available (streaming).")

    for d in qdg.drawings:
        uid = drawing_uid(d)
        if uid in seen:
            continue
        fname = os.path.join(outdir, f"{label}_{saved:03d}.png")
        save_drawing(d, fname)
        seen.add(uid)
        saved += 1
        if saved >= n:
            break

    if saved < n:
        print(f"Warning: dataset exhausted — only saved {saved}/{n} unique drawings for '{label}'.")
    else:
        print(f"Saved {saved} unique drawings for '{label}' in '{outdir}'")

    return saved

def fetch():
    for label in labels:
        label_dir = os.path.join(OUTDIR, label)
        print(f"\nFetching for label: {label}")
        fetch_unique_for_label(label, n=PER_LABEL, outdir=label_dir)

In [47]:
fetch()


Fetching for label: cat
downloading cat from https://storage.googleapis.com/quickdraw_dataset/full/binary/cat.bin
download complete
loading cat drawings
load complete
Label 'cat' has 123202 drawings available (streaming).
Saved 5000 unique drawings for 'cat' in 'quickdraw_images_dedup\cat'

Fetching for label: dog
downloading dog from https://storage.googleapis.com/quickdraw_dataset/full/binary/dog.bin
download complete
loading dog drawings
load complete
Label 'dog' has 152159 drawings available (streaming).
Saved 5000 unique drawings for 'dog' in 'quickdraw_images_dedup\dog'

Fetching for label: van
downloading van from https://storage.googleapis.com/quickdraw_dataset/full/binary/van.bin
download complete
loading van drawings
load complete
Label 'van' has 165909 drawings available (streaming).
Saved 5000 unique drawings for 'van' in 'quickdraw_images_dedup\van'

Fetching for label: swan
downloading swan from https://storage.googleapis.com/quickdraw_dataset/full/binary/swan.bin
downlo